In [14]:
import habitat
from habitat.utils.visualizations import maps
from habitat.utils.visualizations.utils import images_to_video

import os
import numpy as np
import random
import sys
import math

%matplotlib inline
import matplotlib.pyplot as plt

import quaternion

In [15]:
from PIL import Image
from habitat_sim.utils import d3_40_colors_rgb
import numpy as np


def display_sample(rgb_obs, region_maps):
    rgb_img = Image.fromarray(rgb_obs, mode="RGB")
    
    arr = [rgb_img]
    titles = ['rgb']
    
    for room_type, room_map in region_maps.items():
        titles.append(room_type)
        room_img = Image.fromarray(room_map.astype('uint8'))
        arr.append(room_img)
    
    plt.figure(figsize=(12 ,8))
    for i, data in enumerate(arr):
        ax = plt.subplot(1, 6, i+1)
        ax.axis('off')
        ax.set_title(titles[i])
        plt.imshow(data)
    plt.show()
    
def display_sample(rgb_obs, region_maps, top_down):
    rgb_img = Image.fromarray(rgb_obs, mode="RGB")
    
    arr = [rgb_img]
    titles = ['rgb']
    
    for room_type, room_map in region_maps.items():
        titles.append(room_type)
        room_img = Image.fromarray(room_map.astype('uint8'))
        arr.append(room_img)
    
    arr.append(top_down)
    titles.append('top_down')
    
    plt.figure(figsize=(12 ,8))
    for i, data in enumerate(arr):
        ax = plt.subplot(1, 7, i+1)
        ax.axis('off')
        ax.set_title(titles[i])
        plt.imshow(data)
    plt.show()

In [3]:
config = habitat.get_config(config_paths='../configs/tasks/roomnav_mp3d_new.yaml')
config.defrost()
config.DATASET.DATA_PATH = '../data/datasets/roomnav/mp3d/v1/val/val.json.gz'
config.DATASET.SCENES_DIR = '../data/scene_datasets/'
config.SIMULATOR.AGENT_0.SENSORS = ['RGB_SENSOR', 'DEPTH_SENSOR', 'SEMANTIC_SENSOR']
config.SIMULATOR.SEMANTIC_SENSOR.WIDTH = 256
config.SIMULATOR.SEMANTIC_SENSOR.HEIGHT = 256
config.SIMULATOR.TURN_ANGLE = 90
config.freeze()

env = habitat.Env(config=config)
env.episodes = random.sample(env.episodes, 2)

action_mapping = {
    0: 'stop',
    1: 'move_forward',
    2: 'turn left',
    3: 'turn right'
}

2019-07-19 14:42:00,918 initializing sim Sim-v0
I0719 14:42:00.998184 8003 simulator.py:78] Loaded navmesh /private/home/medhini/navigation-analysis-habitat/habitat-api/data/scene_datasets/mp3d/2azQ1b91cZZ/2azQ1b91cZZ.navmesh
2019-07-19 14:42:12,628 initializing task Nav-v0


In [16]:
def draw_top_down_map(info, heading, output_size):
    top_down_map = maps.colorize_topdown_map(info["top_down_map"]["map"])
    original_map_size = top_down_map.shape[:2]
    map_scale = np.array(
        (1, original_map_size[1] * 1.0 / original_map_size[0])
    )
    new_map_size = np.round(output_size * map_scale).astype(np.int32)
    # OpenCV expects w, h but map size is in h, w
    top_down_map = cv2.resize(top_down_map, (new_map_size[1], new_map_size[0]))

    map_agent_pos = info["top_down_map"]["agent_map_coord"]
    map_agent_pos = np.round(
        map_agent_pos * new_map_size / original_map_size
    ).astype(np.int32)
    top_down_map = maps.draw_agent(
        top_down_map,
        map_agent_pos,
        heading - np.pi / 2,
        agent_radius_px=top_down_map.shape[0] / 40,
    )
    return top_down_map

In [17]:
def construct_maps(scene, T_world_curr, crop_size, room_types, scale):
    scene = env.sim.semantic_annotations()
    
    scene_bounds = np.array([scene.aabb.center[0] - scene.aabb.sizes[0]/2, 
                             scene.aabb.center[1] - scene.aabb.sizes[1]/2,
                             scene.aabb.center[2] + scene.aabb.sizes[2]/2,
                             scene.aabb.center[0] + scene.aabb.sizes[0]/2,
                             scene.aabb.center[1] + scene.aabb.sizes[1]/2,
                             scene.aabb.center[2] - scene.aabb.sizes[2]/2
                            ])
    
    T_world_image = np.eye(3)
    T_world_image[0,2] = abs(min(0, scene_bounds[0]))
    T_world_image[1,2] = abs(min(0, scene_bounds[2]))
    
    if scale:
        T_world_image[0,0] = math.floor(100/abs(scene.aabb.sizes[0]))
        T_world_image[1,1] = math.floor(100/abs(scene.aabb.sizes[2]))
    
    region_maps = {}
    
    images = {}
    
    for room_type in room_types:
        if not scale:
            images[room_type] = np.zeros((math.ceil(abs(scene.aabb.sizes[0])), math.ceil(abs(scene.aabb.sizes[2]))))
        else:
            images[room_type] = np.zeros((100,100))
        
    for region in scene.levels[0].regions:
        if region.category.name() in room_types:
            region_center = np.array([region.aabb.center[0], region.aabb.center[1], region.aabb.center[2], 1])
            print('T_world_curr: ', T_world_curr)
            print('Old center: ', region_center)
            
            i,j,k,_ = T_world_curr @ region_center
            
            print('New center: ', [i,j,k])
            
            sys.exit(0)
            
            region_min = np.array([i-region.aabb.sizes[0]/2, k+region.aabb.sizes[2]/2, 1.0])
            region_max = np.array([i+region.aabb.sizes[0]/2, k-region.aabb.sizes[2]/2, 1.0])
            
            region_min_t = T_world_image @ region_min
            region_max_t = T_world_image @ region_max
            
            x1 = math.ceil(region_min_t[0])
            x2 = math.ceil(region_max_t[0])
            y1 = math.ceil(region_min_t[1])
            y2 = math.ceil(region_max_t[1])
            
            if not scale:
                images[region.category.name()][x1:x2,y1:y2] = 255
            
            
    for room_type in room_types:
        img = images[room_type]
        
#         print(room_type)
#         plt.imshow(img)
#         plt.show()
        
        img_padded = np.zeros((1000,1000))
        x1 = math.ceil((1000 - img.shape[0])/2)
        x2 = math.ceil((1000 + img.shape[0])/2)
        y1 = math.ceil((1000 - img.shape[1])/2)
        y2 = math.ceil((1000 + img.shape[1])/2)
        img_padded[x1:x2,y1:y2] = img
        
        x1 = math.ceil((1000 - crop_size)/2)
        x2 = math.ceil((1000 + crop_size)/2)
        y1 = math.ceil((1000 - crop_size)/2)
        y2 = math.ceil((1000 + crop_size)/2)
        cropped_img = img_padded[x1:x2, y1:y2]
        
        region_maps[room_type] = np.asarray(cropped_img)
        
    return region_maps
    

In [18]:
actions = [2,2,2,2]
crop_size = 50
room_types = ['kitchen', 'bedroom', 'bathroom', 'dining room', 'living room']
SCALE = False

for i in range(len(env.episodes)):
    observations = env.reset()
    
    rotation = quaternion.as_rotation_matrix(observations['gps_and_compass'].rot)
    translation = observations['gps_and_compass'].trans
    print('Start Rotation: ', rotation, 'Start Position: ', translation)
    
    T_world_curr = np.eye(4)
    T_world_curr[:3,:3] = rotation
    T_world_curr[:3,3] = translation
    
    scene = env.sim.semantic_annotations()
    region_maps = construct_maps(scene, T_world_curr, crop_size, room_types, SCALE)
    display_sample(observations['rgb'], region_maps)
    
    images = []
    count_steps = 0
    for action in actions:
        observations, _,  _, info = env.step(action)
        
        rotatation = quaternion.as_rotation_matrix(observations['gps_and_compass'].rot)
        translation = observations['gps_and_compass'].trans
        print('Rotation: ', rotatation, 'Position: ', translation)
        
        T_world_curr = np.eye(4)
        T_world_curr[:3,:3] = rotation
        T_world_curr[:3,3] = translation
    
        #Align region maps with new position of agent
        region_maps = construct_maps(scene, T_world_curr, crop_size, room_types, SCALE)
        
        im = observations["rgb"]
        top_down_map = draw_top_down_map(
            info, observations["heading"], im.shape[0]
        )
        output_im = np.concatenate((im, top_down_map), axis=1)
        images.append(output_im)

        display_sample(observations['rgb'], region_maps, output_im)
        
        count_steps += 1
        if env.episode_over:
            print('EPISODE OVER!')
            break

I0719 15:45:21.445246 8003 simulator.py:78] Loaded navmesh /private/home/medhini/navigation-analysis-habitat/habitat-api/data/scene_datasets/mp3d/zsNo4HB9uLZ/zsNo4HB9uLZ.navmesh


Start Rotation:  [[ 0.86095453  0.          0.50868192]
 [ 0.          1.          0.        ]
 [-0.50868192  0.          0.86095453]] Start Position:  [-9.55895221 -0.17162801 13.85466736]
T_world_curr:  [[ 0.86095453  0.          0.50868192 -9.55895221]
 [ 0.          1.          0.         -0.17162801]
 [-0.50868192  0.          0.86095453 13.85466736]
 [ 0.          0.          0.          1.        ]]
Old center:  [-0.40688503  1.39056039 -6.07064915  1.        ]
New center:  [-12.997291158649745, 1.2189323753118515, 8.835089514725858]


SystemExit: 0

In [ ]:
x, y = np.meshgrid(np.linspace(0,99,100), np.linspace(0,99,100))

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(scene_maps[0])

In [ ]:
map = np.zeros((500,500),dtype=np.uint8)
map[:,:] = 255



In [ ]:
T = np.array([[1,0,100],[0,1,100],[0,0,1]])

In [ ]:
img = scene_maps[0]
map = np.zeros((200,200),dtype=np.float32)
map[:,:] = 255.0

plt.figure(figsize=(5,5))
plt.imshow(map)

for i, row in enumerate(img):
    for j, col in enumerate(row):
        pixel_data = img[i,j]
        input_coords = np.array([i,j,1])
        i_out, j_out, _ = T @ input_coords
        if i_out < map.shape[0] and j_out < map.shape[1]:
            map[i_out, j_out] = pixel_data
        
plt.figure(figsize=(5,5))
plt.imshow(map)

In [ ]:
display(floor_map)

In [ ]:
display(floor_map.rotate(270))

In [ ]:
display(floor_map.rotate(90))

In [ ]:
display(floor_map.rotate(180))

In [ ]:
display(floor_map.rotate(300))

In [ ]:
display(floor_map)

In [ ]:
from PIL import Image

img = scene_maps[0]
img = np.asarray(img)
img[:20,:20] = 0.0
img = Image.fromarray(img.astype('uint8'))
a = 1
b = 0
c = 30 #left/right (i.e. 5/-5)
d = 0
e = 1
f = 30 #up/down (i.e. 5/-5)
img = img.transform(img.size, Image.AFFINE, (a, b, c, d, e, f))
display(img)


In [ ]:
img = img.rotate(45)
display(img)